In [1]:
def set_paths():
    '''
    Deployment should be something like "dirs/dir1/use-cases"
    This function adds to the path "dirs/dir1/use-cases" and "dirs/dir1/"
    :return:
    '''
    import imp
    from os.path import dirname
    import os
    import sys

    USE_CASES = "/var/SP/data/home/asaezco/src/devel2/use-cases"#dirname(os.path.abspath(imp.find_module('churn')[1]))
    sys.path.append("/var/SP/data/home/asaezco/src/devel2/pykhaos")
    if USE_CASES not in sys.path:
        sys.path.append(USE_CASES)
        print("Added '{}' to path".format(USE_CASES))

    # if deployment is correct, this path should be the one that contains "use-cases", "pykhaos", ...
    # FIXME another way of doing it more general?
    DEVEL_SRC = os.path.dirname(USE_CASES)  # dir before use-cases dir
    if DEVEL_SRC not in sys.path:
        sys.path.append(DEVEL_SRC)
        print("Added '{}' to path".format(DEVEL_SRC))
set_paths()

Added '/var/SP/data/home/asaezco/src/devel2/use-cases' to path
Added '/var/SP/data/home/asaezco/src/devel2' to path


In [2]:
# coding=utf-8

import sys

from common.src.main.python.utils.hdfs_generic import *
import argparse
import os
import sys
import time
from pyspark.sql.functions import (udf,
                                    col,
                                    decode,
                                    when,
                                    lit,
                                    lower,
                                    concat,
                                    translate,
                                    count,
                                    sum as sql_sum,
                                    max as sql_max,
                                    min as sql_min,
                                    avg as sql_avg,
                                    greatest,
                                    least,
                                    isnull,
                                    isnan,
                                    struct, 
                                    substring,
                                    size,
                                    length,
                                    year,
                                    month,
                                    dayofmonth,
                                    unix_timestamp,
                                    date_format,
                                    from_unixtime,
                                    datediff,
                                    to_date, 
                                    desc,
                                    asc,
                                    countDistinct,
                                    row_number,
                                    skewness,
                                    kurtosis,
                                    concat_ws)

from pyspark.sql import Row, DataFrame, Column, Window
from pyspark.sql.types import DoubleType, StringType, IntegerType, DateType, ArrayType
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer, VectorAssembler, SQLTransformer, OneHotEncoder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from datetime import datetime
from itertools import chain
import numpy as np
from functools import reduce
from utils_general import *
from pykhaos.utils.date_functions import convert_to_date
from utils_model import *
from utils_fbb_churn import *
from metadata_fbb_churn import *
from feature_selection_utils import *
import subprocess


In [3]:
# Para que no haga falta recargar el notebook si hacemos un cambio en alguna de las funciones que estamos importando
%load_ext autoreload
%autoreload 2

In [4]:
# set BDP parameters
def setting_bdp(min_n_executors = 3, max_n_executors = 15, n_cores = 8, executor_memory = "12g", driver_memory="8g",
                   app_name = "Python app", driver_overhead="1g", executor_overhead='3g'):

    MAX_N_EXECUTORS = max_n_executors
    MIN_N_EXECUTORS = min_n_executors
    N_CORES_EXECUTOR = n_cores
    EXECUTOR_IDLE_MAX_TIME = 120
    EXECUTOR_MEMORY = executor_memory
    DRIVER_MEMORY = driver_memory
    N_CORES_DRIVER = 1
    MEMORY_OVERHEAD = N_CORES_EXECUTOR * 2048
    QUEUE = "root.BDPtenants.es.low"
    BDA_CORE_VERSION = "1.0.0"

    SPARK_COMMON_OPTS = os.environ.get('SPARK_COMMON_OPTS', '')
    SPARK_COMMON_OPTS += " --executor-memory %s --driver-memory %s" % (EXECUTOR_MEMORY, DRIVER_MEMORY)
    SPARK_COMMON_OPTS += " --conf spark.shuffle.manager=tungsten-sort"
    SPARK_COMMON_OPTS += "  --queue %s" % QUEUE

    # Dynamic allocation configuration
    SPARK_COMMON_OPTS += " --conf spark.dynamicAllocation.enabled=true"
    SPARK_COMMON_OPTS += " --conf spark.shuffle.service.enabled=true"
    SPARK_COMMON_OPTS += " --conf spark.dynamicAllocation.maxExecutors=%s" % (MAX_N_EXECUTORS)
    SPARK_COMMON_OPTS += " --conf spark.dynamicAllocation.minExecutors=%s" % (MIN_N_EXECUTORS)
    SPARK_COMMON_OPTS += " --conf spark.executor.cores=%s" % (N_CORES_EXECUTOR)
    SPARK_COMMON_OPTS += " --conf spark.dynamicAllocation.executorIdleTimeout=%s" % (EXECUTOR_IDLE_MAX_TIME)
    # SPARK_COMMON_OPTS += " --conf spark.ui.port=58235"
    SPARK_COMMON_OPTS += " --conf spark.port.maxRetries=100"
    SPARK_COMMON_OPTS += " --conf spark.app.name='%s'" % (app_name)
    SPARK_COMMON_OPTS += " --conf spark.submit.deployMode=client"
    SPARK_COMMON_OPTS += " --conf spark.ui.showConsoleProgress=true"
    SPARK_COMMON_OPTS += " --conf spark.sql.broadcastTimeout=1200"
    SPARK_COMMON_OPTS += " --conf spark.yarn.executor.memoryOverhead={}".format(executor_overhead)
    SPARK_COMMON_OPTS += " --conf spark.yarn.executor.driverOverhead={}".format(driver_overhead)

    BDA_ENV = os.environ.get('BDA_USER_HOME', '')

    # Attach bda-core-ra codebase
    SPARK_COMMON_OPTS+=" --files {}/scripts/properties/red_agent/nodes.properties,{}/scripts/properties/red_agent/nodes-de.properties,{}/scripts/properties/red_agent/nodes-es.properties,{}/scripts/properties/red_agent/nodes-ie.properties,{}/scripts/properties/red_agent/nodes-it.properties,{}/scripts/properties/red_agent/nodes-pt.properties,{}/scripts/properties/red_agent/nodes-uk.properties".format(*[BDA_ENV]*7)

    os.environ["SPARK_COMMON_OPTS"] = SPARK_COMMON_OPTS
    os.environ["PYSPARK_SUBMIT_ARGS"] = "%s pyspark-shell " % SPARK_COMMON_OPTS

In [5]:
def get_spark_session(app_name="default name", log_level='INFO', min_n_executors = 3, max_n_executors = 15, n_cores = 4, executor_memory = "12g", driver_memory="8g"):
    HOME_SRC = os.path.join(os.environ.get('BDA_USER_HOME', ''), "src")
    if HOME_SRC not in sys.path:
        sys.path.append(HOME_SRC)


    setting_bdp(app_name=app_name, min_n_executors = min_n_executors, max_n_executors = max_n_executors, n_cores = n_cores, executor_memory = executor_memory, driver_memory=driver_memory)
    from common.src.main.python.utils.hdfs_generic import run_sc
    sc, spark, sql_context = run_sc(log_level=log_level)


    return sc, spark, sql_context

In [6]:
def initialize(app_name, min_n_executors = 3, max_n_executors = 15, n_cores = 4, executor_memory = "12g", driver_memory="8g"):
    import time
    start_time = time.time()

    print("_initialize spark")
    #import pykhaos.utils.pyspark_configuration as pyspark_config
    sc, spark, sql_context = get_spark_session(app_name=app_name, log_level="OFF", min_n_executors = min_n_executors, max_n_executors = max_n_executors, n_cores = n_cores,
                             executor_memory = executor_memory, driver_memory=driver_memory)
    print("Ended spark session: {} secs | default parallelism={}".format(time.time() - start_time,
                                                                         sc.defaultParallelism))
    return spark

In [7]:
spark = initialize("Model Outputs Test ",executor_memory = "32g",min_n_executors = 6,max_n_executors = 15)


_initialize spark
Ended spark session: 234.74130702 secs | default parallelism=2


In [8]:
set_paths()

from pykhaos.utils.date_functions import *
from utils_fbb_churn import *
global sqlContext

In [10]:
df = spark.read.parquet('/data/attributes/vf_es/model_outputs/model_scores/model_name=churn_preds_fbb/year=2019/month=7/day=18')

In [11]:
df.printSchema()

root
 |-- client_id: string (nullable = true)
 |-- scoring: float (nullable = true)
 |-- nif: string (nullable = true)
 |-- msisdn: string (nullable = true)
 |-- executed_at: timestamp (nullable = true)
 |-- model_output: string (nullable = true)
 |-- model_executed_at: timestamp (nullable = true)
 |-- predict_closing_date: string (nullable = true)
 |-- time: integer (nullable = true)
 |-- prediction: string (nullable = true)
 |-- extra_info: string (nullable = true)



In [12]:
df.select('msisdn').show(20)

+------+
|msisdn|
+------+
|     -|
|     -|
|     -|
|     -|
|     -|
|     -|
|     -|
|     -|
|     -|
|     -|
|     -|
|     -|
|     -|
|     -|
|     -|
|     -|
|     -|
|     -|
|     -|
|     -|
+------+
only showing top 20 rows



In [13]:
df.columnss

['client_id',
 'scoring',
 'nif',
 'msisdn',
 'executed_at',
 'model_output',
 'model_executed_at',
 'predict_closing_date',
 'time',
 'prediction',
 'extra_info']

In [14]:
df2 = df.select('client_id',
 'scoring',
 'nif',
 'executed_at',
 'model_output',
 'model_executed_at',
 'predict_closing_date',
 'time',
 'prediction',
 'extra_info')

In [16]:
df_save = df2.withColumn('msisdn', lit(''))

In [17]:
df_save.printSchema()

root
 |-- client_id: string (nullable = true)
 |-- scoring: float (nullable = true)
 |-- nif: string (nullable = true)
 |-- executed_at: timestamp (nullable = true)
 |-- model_output: string (nullable = true)
 |-- model_executed_at: timestamp (nullable = true)
 |-- predict_closing_date: string (nullable = true)
 |-- time: integer (nullable = true)
 |-- prediction: string (nullable = true)
 |-- extra_info: string (nullable = true)
 |-- msisdn: string (nullable = false)



In [20]:
date_name = '20190720'

df_save2 = df_save.withColumn('year', lit(datetime.strptime(date_name, '%Y%m%d').year))\
                   .withColumn('month', lit(datetime.strptime(date_name, '%Y%m%d').month))\
                   .withColumn('day', lit(datetime.strptime(date_name, '%Y%m%d').day))\
        .withColumn('model_name', lit('churn_preds_fbb'))

In [25]:
df_save2.select('model_name').show(10)

+---------------+
|     model_name|
+---------------+
|churn_preds_fbb|
|churn_preds_fbb|
|churn_preds_fbb|
|churn_preds_fbb|
|churn_preds_fbb|
|churn_preds_fbb|
|churn_preds_fbb|
|churn_preds_fbb|
|churn_preds_fbb|
|churn_preds_fbb|
+---------------+
only showing top 10 rows



In [26]:
df_save2.write.partitionBy('model_name', 'year', 'month', 'day').mode("append").format("parquet")\
.save('/data/attributes/vf_es/model_outputs/model_scores/')

In [27]:
spark.stop()

In [ ]:
'/data/attributes/vf_es/model_outputs/model_scores/model_name=churn_preds_fbb/year=2019/month=7/day=18'